In [1]:
import openmeteo_requests
import pandas as pd
import requests_cache
from retry_requests import retry

In [8]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [51]:
cities = {
    'Oslo':
        {"price_area_code": "NO1",
         "longitude": 10.7461,
         "latitude": 59.9127
        },
    'Kristiansand':
        {"price_area_code": "NO2",
         "longitude": 7.9956,
         "latitude": 58.1467
        },
    'Trondheim':
        {"price_area_code": "NO3",
         "longitude": 10.3951,
         "latitude": 63.4305        
        },
    'Tromsø':
        {"price_area_code": "NO4",
         "longitude": 18.9551,
         "latitude": 69.6489        
        },
    'Bergen':
        {"price_area_code": "NO5",
         "longitude": 5.3242,
         "latitude": 60.393        
        }
    }

In [52]:
df = pd.DataFrame(cities)

In [53]:
df

,Oslo,Kristiansand,Trondheim,Tromsø,Bergen
price_area_code,NO1,NO2,NO3,NO4,NO5
longitude,10.7461,7.9956,10.3951,18.9551,5.3242
latitude,59.9127,58.1467,63.4305,69.6489,60.393


In [25]:
responses = openmeteo.weather_api(url, params=params)

In [61]:
def create_dates(year):
    start = f'{year}-01-01'
    end = f'{year}-12-31'
    return start, end

In [62]:
def fetch_data(longitude, latitude, year):
    start, end = create_dates(year)
    params = {
	"latitude": latitude,
	"longitude": longitude,
    "start_date": start,
    "end_date": end,
	"hourly": "temperature_2m",
    }
    
    responses = openmeteo.weather_api(url, params=params)

In [63]:
year = 2019
res = fetch_data(df.Bergen.longitude, df.Bergen.latitude, year)

OpenMeteoRequestsError: failed to request 'https://api.open-meteo.com/v1/forecast': {'error': True, 'reason': "Parameter 'start_date' is out of allowed range from 2025-08-02 to 2025-11-18"}

In [27]:
response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

Coordinates: 69.64802551269531°N 18.943801879882812°E
Elevation: 14.0 m asl
Timezone difference to GMT+0: 0s
